In [1]:
from splinter import Browser
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [3]:
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"} # MAC USERS!
executable_path = "chromedriver.exe"
# executable_path = {"executable_path": "chromedriver.exe"} # WINDOWS USERS!
# browser = Browser("chrome", **executable_path, headless=False)

In [4]:
driver = webdriver.Chrome(executable_path = executable_path)
driver.implicitly_wait(2)

In [5]:
info = {}
url = 'https://data.bls.gov/lausmap/showMap.jsp'
response = driver.get(url)

In [21]:
columns = []
for x in range(6):
    dropdown = Select(driver.find_element_by_id('period'))
    dropdown.select_by_index(x)
    search_button = driver.find_element_by_id('btn_sumbit')
    search_button.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    states = str(soup.find_all("table", id = "tb_data")[0])
    states_df = pd.read_html(states)[0]
    if x == 0:
        state_col = states_df.iloc[:,0]
        unemp_col = states_df.iloc[:,1]
        unemp_df = pd.concat([state_col, unemp_col], axis = 1)
        columns.append(states_df.columns.values[1])
    else:
        new_col = str(states_df.columns.values[1])
        unemp_df[new_col] = states_df.iloc[:,1]
        columns.append(new_col)
print(columns)
unemp_df.head()

['January2020', 'February2020', 'March2020', 'April2020', 'May2020', 'June2020']


,County,January2020,February2020,March2020,April2020,May2020,June2020
0,Autauga County,3.0,2.7,2.8,11.4,7.8,6.6
1,Baldwin County,3.1,2.6,2.7,15.4,9.4,7.3
2,Barbour County,3.8,3.5,3.7,9.9,8.7,8.9
3,Bibb County,3.3,2.9,3.1,17.7,10.7,8.6
4,Blount County,3.0,2.5,2.7,9.4,6.4,5.7


In [11]:
record = unemp_df.to_json(orient = 'records')
parsed = json.loads(record)
print(json.dumps(parsed, indent=4, sort_keys=True))


[
    {
        "April2020": "13.8",
        "February2020": 2.7,
        "January2020": 2.7,
        "June2020": "7.5",
        "March2020": "3.0",
        "May2020": "9.6",
        "State": "Alabama"
    },
    {
        "April2020": "13.5",
        "February2020": 5.8,
        "January2020": 6.0,
        "June2020": "12.4",
        "March2020": "5.2",
        "May2020": "12.7",
        "State": "Alaska"
    },
    {
        "April2020": "13.4",
        "February2020": 4.5,
        "January2020": 4.5,
        "June2020": "10.0",
        "March2020": "6.1",
        "May2020": "9.0",
        "State": "Arizona"
    },
    {
        "April2020": "10.8",
        "February2020": 3.5,
        "January2020": 3.5,
        "June2020": "8.0",
        "March2020": "5.0",
        "May2020": "9.6",
        "State": "Arkansas"
    },
    {
        "April2020": "16.4",
        "February2020": 3.9,
        "January2020": 3.9,
        "June2020": "14.9",
        "March2020": "5.5",
        "May2020": 

In [14]:
st_tab = driver.find_element_by_id('tab_county_u')
st_tab.click()
time.sleep(2)
for state in range(50):
    st_dropdown = driver.find_element_by_id('state')
    st_dropdown_sel = Select(st_dropdown)
    st_dropdown_sel.select_by_index(state)
    state_col = st_dropdown_sel.first_selected_option.text
    for x in range(6):
        dropdown = Select(driver.find_element_by_id('period'))
        dropdown.select_by_index(x)
        search_button = driver.find_element_by_id('btn_sumbit')
        search_button.click()
        time.sleep(1)
        county_soup = BeautifulSoup(driver.page_source, 'lxml')
        counties = str(county_soup.find_all("table", id = "tb_data")[0])
        counties_df = pd.read_html(counties)[0]
        if state == 0:
            if x == 0:
                county_col = counties_df.iloc[:,0]
                ct_unemp_col = counties_df.iloc[:,1]
                unemp_by_county_df = pd.concat([county_col, ct_unemp_col], axis = 1)
                unemp_by_county_df['State'] = state_col
            else:
                new_col = str(counties_df.columns.values[1])
                unemp_by_county_df[new_col] = counties_df.iloc[:,1]
        else:
            if x == 0:
                county_col = counties_df.iloc[:,0]
                ct_unemp_col = counties_df.iloc[:,1]
                ct_unemp_df = pd.concat([county_col, ct_unemp_col], axis = 1)
            else:
                new_col = str(counties_df.columns.values[1])
                ct_unemp_df[new_col] = counties_df.iloc[:,1]
    ct_unemp_df['State'] = state_col
    unemp_by_county_df = pd.concat([unemp_by_county_df, ct_unemp_df], axis = 0)
    print(unemp_by_county_df)

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



    April2020             County  February2020  January2020  June2020  \
0        11.4     Autauga County           2.7          3.0       6.6   
1        15.4     Baldwin County           2.6          3.1       7.3   
2         9.9     Barbour County           3.5          3.8       8.9   
3        17.7        Bibb County           2.9          3.3       8.6   
4         9.4      Blount County           2.5          3.0       5.7   
..        ...                ...           ...          ...       ...   
62        NaN  Tuscaloosa County           NaN          NaN       NaN   
63        NaN      Walker County           NaN          NaN       NaN   
64        NaN  Washington County           NaN          NaN       NaN   
65        NaN      Wilcox County           NaN          NaN       NaN   
66        NaN     Winston County           NaN          NaN       NaN   

    March2020  May2020    State  
0         2.8      7.8  Alabama  
1         2.7      9.4  Alabama  
2         3.7      8.

    April2020             County  February2020  January2020  June2020  \
0        11.4     Autauga County           2.7          3.0       6.6   
1        15.4     Baldwin County           2.6          3.1       7.3   
2         9.9     Barbour County           3.5          3.8       8.9   
3        17.7        Bibb County           2.9          3.3       8.6   
4         9.4      Blount County           2.5          3.0       5.7   
..        ...                ...           ...          ...       ...   
6         6.5     Tolland County           3.9          4.0       7.9   
7         8.1     Windham County           4.8          4.8       9.6   
0        15.8        Kent County           4.4          4.5      14.1   
1        14.1  New Castle County           3.8          4.0      12.8   
2        16.0      Sussex County           4.6          4.8      11.5   

    March2020  May2020        State  
0         2.8      7.8      Alabama  
1         2.7      9.4      Alabama  
2        

    April2020             County  February2020  January2020  June2020  \
0        11.4     Autauga County           2.7          3.0       6.6   
1        15.4     Baldwin County           2.6          3.1       7.3   
2         9.9     Barbour County           3.5          3.8       8.9   
3        17.7        Bibb County           2.9          3.3       8.6   
4         9.4      Blount County           2.5          3.0       5.7   
..        ...                ...           ...          ...       ...   
94       17.2   Winnebago County           2.9          3.2      10.1   
95       11.2  Winneshiek County           4.0          5.1       7.9   
96       10.2    Woodbury County           3.2          3.6       8.1   
97       14.0       Worth County           3.5          4.2       7.9   
98        7.4      Wright County           2.9          3.1       5.9   

    March2020  May2020    State  
0         2.8      7.8  Alabama  
1         2.7      9.4  Alabama  
2         3.7      8.

    April2020            County  February2020  January2020  June2020  \
0        11.4    Autauga County           2.7          3.0       6.6   
1        15.4    Baldwin County           2.6          3.1       7.3   
2         9.9    Barbour County           3.5          3.8       8.9   
3        17.7       Bibb County           2.9          3.3       8.6   
4         9.4     Blount County           2.5          3.0       5.7   
..        ...               ...           ...          ...       ...   
78       32.0    Tuscola County           5.1          6.0      14.4   
79       20.9  Van Buren County           4.5          5.3      12.4   
80       14.8  Washtenaw County           2.2          2.5      10.5   
81       25.8      Wayne County           4.4          5.1      20.7   
82       31.3    Wexford County           4.6          5.4      14.3   

    March2020  May2020     State  
0         2.8      7.8   Alabama  
1         2.7      9.4   Alabama  
2         3.7      8.7   Alaba

    April2020               County  February2020  January2020  June2020  \
0        11.4       Autauga County           2.7          3.0       6.6   
1        15.4       Baldwin County           2.6          3.1       7.3   
2         9.9       Barbour County           3.5          3.8       8.9   
3        17.7          Bibb County           2.9          3.3       8.6   
4         9.4        Blount County           2.5          3.0       5.7   
..        ...                  ...           ...          ...       ...   
5        17.2  Hillsborough County           3.2          3.1      12.0   
6        15.7     Merrimack County           2.6          2.7      10.8   
7        17.6    Rockingham County           3.3          3.3      11.6   
8        16.4     Strafford County           2.8          2.7      11.4   
9        13.6      Sullivan County           2.7          2.8       9.6   

    March2020  May2020          State  
0         2.8      7.8        Alabama  
1         2.7      

    April2020             County  February2020  January2020  June2020  \
0        11.4     Autauga County           2.7          3.0       6.6   
1        15.4     Baldwin County           2.6          3.1       7.3   
2         9.9     Barbour County           3.5          3.8       8.9   
3        17.7        Bibb County           2.9          3.3       8.6   
4         9.4      Blount County           2.5          3.0       5.7   
..        ...                ...           ...          ...       ...   
72       13.9     Wagoner County           3.0          3.2       6.6   
73       13.1  Washington County           3.1          3.2       6.6   
74       12.5     Washita County           3.8          4.1       7.0   
75        6.6       Woods County           1.7          1.9       5.5   
76       12.6    Woodward County           3.1          3.6       7.2   

    March2020  May2020     State  
0         2.8      7.8   Alabama  
1         2.7      9.4   Alabama  
2         3.7     

     April2020          County  February2020  January2020  June2020  \
0         11.4  Autauga County           2.7          3.0       6.6   
1         15.4  Baldwin County           2.6          3.1       7.3   
2          9.9  Barbour County           3.5          3.8       8.9   
3         17.7     Bibb County           2.9          3.3       8.6   
4          9.4   Blount County           2.5          3.0       5.7   
..         ...             ...           ...          ...       ...   
249       10.4     Wood County           4.1          4.3       7.6   
250       16.4   Yoakum County           3.5          3.5      14.7   
251        8.3    Young County           3.0          3.2       6.1   
252       13.7   Zapata County           6.2          6.5      13.5   
253       17.6   Zavala County           9.0          9.2      17.0   

     March2020  May2020    State  
0          2.8      7.8  Alabama  
1          2.7      9.4  Alabama  
2          3.7      8.7  Alabama  
3      

    April2020             County  February2020  January2020  June2020  \
0        11.4     Autauga County           2.7          3.0       6.6   
1        15.4     Baldwin County           2.6          3.1       7.3   
2         9.9     Barbour County           3.5          3.8       8.9   
3        17.7        Bibb County           2.9          3.3       8.6   
4         9.4      Blount County           2.5          3.0       5.7   
..        ...                ...           ...          ...       ...   
18       10.5  Sweetwater County           5.4          5.4       9.4   
19       18.2       Teton County           2.6          2.7       9.3   
20        9.4       Uinta County           4.6          4.7       8.5   
21        7.4    Washakie County           5.0          4.6       6.6   
22        5.7      Weston County           3.2          3.1       5.0   

    March2020  May2020    State  
0         2.8      7.8  Alabama  
1         2.7      9.4  Alabama  
2         3.7      8.

,County,January2020,February2020,March2020,April2020,May2020,June2020,State
0,Albany County,3.3,3.1,3.1,5.4,4.9,4.5,Wyoming
1,Big Horn County,5.4,5.5,5.5,6.8,5.8,5.6,Wyoming
2,Campbell County,3.9,3.8,4.1,10.2,9.8,9.5,Wyoming
3,Carbon County,4.2,4.4,4.3,6.8,7.0,5.8,Wyoming
4,Converse County,3.0,3.1,3.3,6.4,7.3,7.3,Wyoming


In [20]:
unemp_by_county_df = unemp_by_county_df[['State','County','January2020','February2020','March2020','April2020','May2020','June2020']]
unemp_by_county_df

,State,County,January2020,February2020,March2020,April2020,May2020,June2020
0,Alabama,Autauga County,3.0,2.7,2.8,11.4,7.8,6.6
1,Alabama,Baldwin County,3.1,2.6,2.7,15.4,9.4,7.3
2,Alabama,Barbour County,3.8,3.5,3.7,9.9,8.7,8.9
3,Alabama,Bibb County,3.3,2.9,3.1,17.7,10.7,8.6
4,Alabama,Blount County,3.0,2.5,2.7,9.4,6.4,5.7
...,...,...,...,...,...,...,...,...
18,Wyoming,Sweetwater County,5.4,5.4,5.6,10.5,9.7,9.4
19,Wyoming,Teton County,2.7,2.6,2.9,18.2,15.0,9.3
20,Wyoming,Uinta County,4.7,4.6,4.8,9.4,8.9,8.5
21,Wyoming,Washakie County,4.6,5.0,5.3,7.4,7.1,6.6


In [22]:
county_record = unemp_by_county_df.to_json(orient = 'records')
county_parsed = json.loads(county_record)
print(json.dumps(county_parsed, indent=4, sort_keys=True))

[
    {
        "April2020": 11.4,
        "County": "Autauga County",
        "February2020": 2.7,
        "January2020": 3.0,
        "June2020": 6.6,
        "March2020": 2.8,
        "May2020": 7.8,
        "State": "Alabama"
    },
    {
        "April2020": 15.4,
        "County": "Baldwin County",
        "February2020": 2.6,
        "January2020": 3.1,
        "June2020": 7.3,
        "March2020": 2.7,
        "May2020": 9.4,
        "State": "Alabama"
    },
    {
        "April2020": 9.9,
        "County": "Barbour County",
        "February2020": 3.5,
        "January2020": 3.8,
        "June2020": 8.9,
        "March2020": 3.7,
        "May2020": 8.7,
        "State": "Alabama"
    },
    {
        "April2020": 17.7,
        "County": "Bibb County",
        "February2020": 2.9,
        "January2020": 3.3,
        "June2020": 8.6,
        "March2020": 3.1,
        "May2020": 10.7,
        "State": "Alabama"
    },
    {
        "April2020": 9.4,
        "County": "Blount Cou